In [79]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
from keras.layers import Bidirectional,LSTM,Dense,Embedding,Dropout,Activation,Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import gensim
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Conv1D, MaxPooling1D
from keras.models import load_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from keras import backend as K
from keras.layers import Layer,concatenate,BatchNormalization,GlobalMaxPooling1D,GlobalAveragePooling1D


In [2]:
df=pd.read_csv('../input/sentiment140/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=["target", "ids", "date", "flag", "user", "text"])

In [3]:
df.head(5)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
nltk.download('stopwords')
stop_words=stopwords.words('english')
stemmer=SnowballStemmer('english')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def preprocessing(text):
    pattern="@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    text=re.sub(pattern," ",str(text).lower()).strip()
    tokens=[]
    for token in text.split():
#         tokens.append(token)
        if token not in stop_words:
            tokens.append(stemmer.stem(token))
    return " ".join(tokens)

In [9]:
df.text=df.text.apply(lambda x: preprocessing(x))

In [10]:
df_train,df_test=train_test_split(df,test_size=0.05,random_state=1)
len(df_test)

80000

In [11]:
documents=[t.split() for t in df_train.text]
documents

[['time', 'girl', 'ur', 'go', 'jb', 'right', 'show'],
 ['b', 'wait', 'come', 'till', '9', 'good', 'luck', 'surviv', 'faker', 'lol'],
 ['haha', 'yeah', 'sound', 'right'],
 ['realiz',
  'download',
  'iphon',
  '3',
  '0',
  'softwar',
  'jailbroken',
  'app',
  'gone'],
 ['take', 'take', 'www', 'thepulseofentertain', 'com'],
 ['laundri', 'duti', 'got', 'get', 'round', 'write', 'cv', 'point'],
 ['aww', 'thank'],
 ['want', 'school', 'day', 'alreadi', 'andi', 'come', 'home', 'today'],
 ['look', 'forward', 'next', 'monday'],
 ['homework'],
 ['crabtre', 'seth', 'amp', 'noah'],
 ['sound', 'like', 'simpli', 'wonder', 'way', 'spend', 'sunday', 'enjoy'],
 ['night', 'u'],
 ['hangin', 'fam', 'head', 'hurtin'],
 ['texa'],
 ['shall', 'loveeee', 'x', 'x', 'x'],
 ['jersey', 'til', 'tuesday', 'holla', 'twitter'],
 ['rain', 'conduc', 'boat', 'parti', 'plan'],
 ['hahaha',
  'im',
  'total',
  'consid',
  'like',
  'shark',
  'im',
  'sure',
  'what',
  'ur',
  'fav',
  'anim'],
 ['ive',
  'read',
  'stea

In [12]:
w2v_model=gensim.models.word2vec.Word2Vec(size=300,window=5,min_count=10,workers=8)

In [13]:
w2v_model.build_vocab(documents)

In [14]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 26091


In [15]:
w2v_model.train(documents, total_examples=len(documents), epochs=32)

(308609076, 350743872)

In [16]:
w2v_model.most_similar("love")


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('amaz', 0.5878851413726807),
 ('luv', 0.5642381906509399),
 ('awesom', 0.5115134716033936),
 ('ador', 0.48541098833084106),
 ('loov', 0.4713866114616394),
 ('looov', 0.45746463537216187),
 ('loveeee', 0.44606027007102966),
 ('loooov', 0.44229575991630554),
 ('great', 0.43080195784568787),
 ('looooov', 0.42890632152557373)]

In [17]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df_train.text)


In [25]:
X_train=pad_sequences(tokenizer.texts_to_sequences(df_train.text),maxlen=280)
X_test=pad_sequences(tokenizer.texts_to_sequences(df_test.text),maxlen=280)

In [26]:
X_train.shape

(1520000, 280)

In [20]:
decode_map = {0: "NEGATIVE", 2: "NEUTRAL", 4: "POSITIVE"}
def decode_sentiment(label):
    return decode_map[int(label)]
df_train.target = df_train.target.apply(lambda x: decode_sentiment(x))
df_test.target = df_test.target.apply(lambda x: decode_sentiment(x))


/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [21]:
labels=list(df_train.target.unique())
labels

['POSITIVE', 'NEGATIVE']

In [22]:
encoder=LabelEncoder()
y_train=encoder.fit_transform(df_train.target).reshape(-1,1)
y_test=encoder.transform(df_test.target).reshape(-1,1)

In [23]:
vocab_size = len(tokenizer.word_index) + 1

y_train.shape


(1520000, 1)

In [27]:
embedding_matrix=np.zeros((vocab_size,300))
for word,i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i]=w2v_model.wv[word]

In [28]:
embedding_layer=Embedding(vocab_size,300,weights=[embedding_matrix],input_length=280,trainable=False)

In [29]:
bs=1024

callbacks=[
    EarlyStopping(
        patience=4,
        monitor='val_accuracy',
    ),
    
    ReduceLROnPlateau(monitor='loss',
                     factor=0.1,
                     patience=2,
                     cooldown=2,
                     verbose=1)
]
lr_decay = (1./0.8 -1)/bs
opt = Adam(lr=1e-3,beta_1=0.9,beta_2=0.999,decay=0.01)


In [30]:
X_input=Input(shape=(X_train.shape[1],))
X=embedding_layer(X_input)
X=Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(X)
output=Dense(1,activation='sigmoid')(X)
model=Model(inputs=X_input,outputs=output)
              

In [31]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 280)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 280, 300)          83661000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 84,100,553
Trainable params: 439,553
Non-trainable params: 83,661,000
_________________________________________________________________


In [32]:
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [33]:
model.fit(X_train,y_train, epochs=2, batch_size=bs,validation_split=0.05,callbacks=callbacks)

Train on 1444000 samples, validate on 76000 samples
Epoch 1/2
1444000/1444000 [==============================] - 682s 472us/step - loss: 0.4826 - accuracy: 0.7664 - val_loss: 0.4677 - val_accuracy: 0.7742
Epoch 2/2
1444000/1444000 [==============================] - 680s 471us/step - loss: 0.4699 - accuracy: 0.7741 - val_loss: 0.4642 - val_accuracy: 0.7764


In [34]:

score = model.evaluate(X_test, y_test, batch_size=512)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

80000/80000 [==============================] - 15s 184us/step

ACCURACY: 0.779574990272522
LOSS: 0.46106054406166075


In [35]:
X_input=Input(shape=(X_train.shape[1],))
X=embedding_layer(X_input)
X=Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))(X)
X=Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(X)
X=Dense(32,activation='relu')(X)
X=Dense(32,activation='relu')(X)
output=Dense(1,activation='sigmoid')(X)
model2=Model(inputs=X_input,outputs=output)

In [36]:
model2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 280)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 280, 300)          83661000  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 280, 256)          439296    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               164352    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33  

In [37]:
model2.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [38]:
model2.fit(X_train,y_train, epochs=2, batch_size=bs,validation_split=0.05,callbacks=callbacks)

Train on 1444000 samples, validate on 76000 samples
Epoch 1/2
1444000/1444000 [==============================] - 1467s 1ms/step - loss: 0.5001 - accuracy: 0.7538 - val_loss: 0.4781 - val_accuracy: 0.7684
Epoch 2/2
1444000/1444000 [==============================] - 1487s 1ms/step - loss: 0.4828 - accuracy: 0.7656 - val_loss: 0.4741 - val_accuracy: 0.7718


In [39]:
score = model2.evaluate(X_test, y_test, batch_size=1024)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

80000/80000 [==============================] - 19s 233us/step

ACCURACY: 0.7735000252723694
LOSS: 0.4713328702449799


In [41]:
X_input=Input(shape=(X_train.shape[1],))
X=embedding_layer(X_input)
X=Conv1D(250,3,padding='valid',activation='relu',strides=1)(X)
X=MaxPooling1D()(X)
X=Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2))(X)
output=Dense(1,activation='sigmoid')(X)
model3=Model(inputs=X_input,outputs=output)

In [42]:
model3.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 280)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 280, 300)          83661000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 278, 250)          225250    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 139, 250)          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               388096    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 84,274,603
Trainable params: 613,603
Non-trainable params: 83,661,000
_________________________________________

In [43]:
# opt = Adam(lr=0.005,beta_1=0.9,beta_2=0.999,decay=0.01)
model3.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [44]:
model3.fit(X_train,y_train, epochs=2, batch_size=bs,validation_split=0.05,callbacks=callbacks)

Train on 1444000 samples, validate on 76000 samples
Epoch 1/2
1444000/1444000 [==============================] - 481s 333us/step - loss: 0.5096 - accuracy: 0.7484 - val_loss: 0.4878 - val_accuracy: 0.7615
Epoch 2/2
1444000/1444000 [==============================] - 476s 330us/step - loss: 0.4872 - accuracy: 0.7628 - val_loss: 0.4818 - val_accuracy: 0.7652


In [45]:
score = model3.evaluate(X_test, y_test, batch_size=1024)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

80000/80000 [==============================] - 7s 92us/step

ACCURACY: 0.7673749923706055
LOSS: 0.4793182584762573


In [48]:
def predict(text):
    
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=280)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score)

    return {"label": label, "score": float(score)}  

In [51]:
predict("I hate the music")

{'label': 'NEGATIVE', 'score': 0.11266549676656723}

In [ ]:
model.save('model.h5')
model2.save('model2.h5')
model3.save('model3.h5')
w2v_model.save('model.w2v')

In [96]:
class AvgWords(Layer):
    def __init__(self,output_dim=300):
        super(AvgWords,self).__init__()
        self.output_dim=output_dim

    def call(self, x):
        axis = K.ndim(x) - 2
        return K.mean(x, axis=axis)

    def compute_output_shape(self,input_shape):
        return (input_shape[0], self.output_dim)

In [92]:
class WordDropout(Layer):
    def __init__(self,rate):
        super(WordDropout,self).__init__()
        self.rate=rate

    def call(self,inputs):
        input_shape = K.shape(inputs)
        batch_size = input_shape[0]
        words = input_shape[1]
        mask = K.random_uniform((batch_size, words, 1)) >= self.rate
        w_drop = K.cast(mask, 'float32') * inputs
        return w_drop


In [97]:
no_hidden_layers=5
X_input=Input(shape=(X_train.shape[1],))
X1=embedding_layer(X_input)
X1=WordDropout(0.2)(X1)
X1=AvgWords()(X1)
# X1=GlobalAveragePooling1D()(X1)
for i in range(no_hidden_layers):
    X1=Dense(300,activation='tanh')(X1)
    X1=BatchNormalization()(X1)
    X1=Dropout(0.6)(X1)
# X1=Dense(300,activation='relu')(X1)
# X1=Dense(300,activation='relu')(X1)
out=Dense(1,activation='sigmoid')(X1)
model4=Model(inputs=X_input,outputs=out)
model4.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 280)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 280, 300)          83661000  
_________________________________________________________________
word_dropout_12 (WordDropout (None, 280, 300)          0         
_________________________________________________________________
avg_words_12 (AvgWords)      (None, 300)               0         
_________________________________________________________________
dense_97 (Dense)             (None, 300)               90300     
_________________________________________________________________
batch_normalization_73 (Batc (None, 300)               1200      
_________________________________________________________________
dropout_73 (Dropout)         (None, 300)               0  

In [98]:
model4.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
model4.fit(X_train,y_train, epochs=20, batch_size=bs,validation_split=0.05,callbacks=callbacks)

Train on 1444000 samples, validate on 76000 samples
Epoch 1/20
1444000/1444000 [==============================] - 19s 13us/step - loss: 0.6920 - accuracy: 0.5967 - val_loss: 0.6529 - val_accuracy: 0.6967
Epoch 2/20
1444000/1444000 [==============================] - 19s 13us/step - loss: 0.6050 - accuracy: 0.6758 - val_loss: 0.6701 - val_accuracy: 0.7126
Epoch 3/20
1444000/1444000 [==============================] - 17s 12us/step - loss: 0.5838 - accuracy: 0.6968 - val_loss: 0.6717 - val_accuracy: 0.7210
Epoch 4/20
1444000/1444000 [==============================] - 17s 12us/step - loss: 0.5744 - accuracy: 0.7060 - val_loss: 0.6675 - val_accuracy: 0.7228
Epoch 5/20
1444000/1444000 [==============================] - 18s 13us/step - loss: 0.5693 - accuracy: 0.7105 - val_loss: 0.6630 - val_accuracy: 0.7270
Epoch 6/20
1444000/1444000 [==============================] - 18s 12us/step - loss: 0.5658 - accuracy: 0.7138 - val_loss: 0.6613 - val_accuracy: 0.7259
Epoch 7/20
1444000/1444000 [========

In [ ]:
score = model4.evaluate(X_test, y_test, batch_size=1024)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])